__COMSW4121 Homework 6: Spark Homework__ 


Patrick Rogan

UNI: psr2125

In [1]:
sc

In [2]:
# All files copied to vagrant vm by vagrant scp
# moved file1 and file1 to /home/vagrant/P1 in order to be able to use wholeTextFiles
lines = sc.textFile("P1/file1")
lines

P1/file1 MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [3]:
lines.collect()

[u'Hello World Bye World']

__Total Word Count__

In [4]:
words = lines.flatMap(lambda l: l.split(" "))

In [5]:
words.count()

4

__Problem 1__

In [6]:
f1f2 = sc.wholeTextFiles("P1/",use_unicode = False)

In [7]:
phrases = f1f2.map(lambda (x,y):y.strip())

In [8]:
import re
wc = phrases.flatMap(lambda l: l.split(" ")).map(lambda l: (l,1)).reduceByKey(lambda x,y: x + y)

for w in wc.collect():
    print(re.sub(',',':', re.sub('[\(\)\']','',str(w))))

World: 2
Bye: 1
Hello: 2
Goodbye: 1
Hadoop: 2


__Problem 2__

In [9]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

In [10]:
lines = sc.textFile("purchases.txt")

In [11]:
# convert each entry of the RDD into a tuple containing values of each field in correct format
splitLines = lines.map(lambda ln: ln.split(","))
purch = splitLines.map(lambda wd: (wd[0], wd[1], wd[2], float(wd[3]), wd[4].strip()))

# encode the schema into an array called fields
fields = [StructField('timestamp', StringType(), True),
          StructField('location', StringType(), True),
          StructField('category', StringType(), True), 
          StructField('price', FloatType(), True),
          StructField('card', StringType(), True)]

schema = StructType(fields)

# apply the schema to the RDD
schemaPurchases = sqlContext.createDataFrame(purch, schema)

# register the DataFrame as a table called purchases
schemaPurchases.registerTempTable("purchases")

In [12]:
# i. What is the average price of the products that were purchased via Mastercard?
sqlContext.sql("SELECT AVG(price) FROM purchases WHERE card = 'MasterCard'").show()

c0               
275.0677317417774


In [13]:
# ii. Which date recorded the highest total sales?
sqlContext.sql("SELECT SUBSTR(timestamp,1,10) as SS,SUM(price) as SP from purchases GROUP BY SUBSTR(timestamp,1,10) ORDER BY SP DESC LIMIT 1").show()

SS         SP               
2012-03-17 2384.480026245117


In [14]:
# iii. What is the minimum value of a product under the Computers category?
sqlContext.sql("SELECT price FROM purchases WHERE category = 'Computers' ORDER BY price LIMIT 1").show()

price
0.38 


In [15]:
# iv. How many distinct categories of products are there?
sqlContext.sql("SELECT COUNT(DISTINCT(category)) FROM purchases").show()

c0
18


In [16]:
# v. Which store location had the lowest total sales?
sqlContext.sql("SELECT location, SUM(price) AS SP FROM purchases GROUP by location ORDER BY SP LIMIT 1").show()

location SP               
Plano    784.9599838256836


__Problem 3__

In [17]:
purch.take(5)

[(u'2012-07-20 09:59:00', u'Corpus Christi', u'CDs', 327.91, u'Cash'),
 (u'2012-03-11 17:29:00', u'Durham', u'Books', 115.09, u'Discover'),
 (u'2012-07-31 11:43:00', u'Rochester', u'Toys', 332.07, u'MasterCard'),
 (u'2012-06-18 14:47:00', u'Garland', u'Computers', 31.99, u'Visa'),
 (u'2012-03-27 11:40:00', u'Tulsa', u'CDs', 452.18, u'Discover')]

In [18]:
purchasesDF = purch.toDF(['timestamp','location','category','price','card'])
purchasesDF.show(5)

timestamp           location       category  price  card      
2012-07-20 09:59:00 Corpus Christi CDs       327.91 Cash      
2012-03-11 17:29:00 Durham         Books     115.09 Discover  
2012-07-31 11:43:00 Rochester      Toys      332.07 MasterCard
2012-06-18 14:47:00 Garland        Computers 31.99  Visa      
2012-03-27 11:40:00 Tulsa          CDs       452.18 Discover  


In [19]:
from pyspark.sql.functions import *

In [20]:
# i. What is the average price of the products that were purchased via Mastercard?
purchasesDF.filter(purchasesDF.card == "MasterCard").agg(avg(col('price'))).show()

AVG(price)        
275.06773195876286


In [21]:
# ii. Which date recorded the highest total sales?
#purchasesDF.select('timestamp','price').groupBy('timestamp').agg(sum('price')).show()

In [22]:
# iii. What is the minimum value of a product under the Computers category?
purchasesDF.filter(purchasesDF.category == "Computers").agg(min(col('price'))).show()

MIN(price)
0.38      


In [23]:
# iv. How many distinct categories of products are there?
cats = purchasesDF.select('category').groupBy('category').agg(countDistinct('category'))
print(len(cats.collect()))

18


In [24]:
# v. Which store location had the lowest total sales?
#purchasesDF.select('location','price').groupBy('location').agg(sum('price')).show()